# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes. 


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever). 


### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.

### First, download the data from: https://www.kaggle.com/ntnu-testimon/paysim1. Import the dataset and provide some discriptive statistics and plots. What do you think will be the important features in determining the outcome?
### Note: don't use the entire dataset, use a sample instead, with n=100000 elements, so your computer doesn't freeze.

In [2]:
# Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler, PolynomialFeatures, MinMaxScaler
import seaborn as sns

fraud = pd.read_csv('PS_20174392719_1491204439457_log.csv').sample(1000000)

In [3]:
fraud.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
5118179,356,PAYMENT,4177.56,C1840332845,0.0,0.0,M2103313381,0.00,0.00,0,0
5944906,405,PAYMENT,2472.53,C950533811,69.0,0.0,M687355628,0.00,0.00,0,0
42393,9,CASH_OUT,10362.80,C650625657,30728.0,20365.2,C1545688642,77114.12,87476.92,0,0
680604,36,TRANSFER,980285.34,C1146185128,0.0,0.0,C1881694426,4139918.73,5120204.07,0,0
4201367,305,CASH_OUT,208903.27,C1534395067,0.0,0.0,C117010065,6379210.56,6588113.83,0,0


In [4]:
fraud.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [6]:
fraud['type'].value_counts()

CASH_OUT    351488
PAYMENT     338120
CASH_IN     219507
TRANSFER     84363
DEBIT         6522
Name: type, dtype: int64

In [7]:
fraud['nameDest'].value_counts()

C1286084959    23
C97730845      21
C1023714065    20
C998351292     20
C985934102     19
               ..
M945469419      1
C882836013      1
C706667044      1
M367906740      1
M40886845       1
Name: nameDest, Length: 648315, dtype: int64

In [8]:
fraud['nameOrig'].value_counts()

C43341574      2
C1321614702    2
C400960377     2
C691649680     2
C257315024     2
              ..
C895869469     1
C526649453     1
C2134337981    1
C1131685303    1
C527156172     1
Name: nameOrig, Length: 999749, dtype: int64

In [ ]:
'''I will drop namedest and nameorig, because there are too many unique values '''

In [9]:
fraud.drop(labels=['nameDest', 'nameOrig'], axis=1, inplace=True)

### What is the distribution of the outcome? 

In [11]:
# Your response here
fraud['isFraud'].value_counts(normalize=True)

0    0.998626
1    0.001374
Name: isFraud, dtype: float64

### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [12]:
# Your code here
le = LabelEncoder()
label_cols = ['type']
fraud[label_cols] = fraud[label_cols].apply(le.fit_transform)

### Run a logisitc regression classifier and evaluate its accuracy.

In [13]:
# Your code here
y = fraud['isFraud']
X = fraud.drop(labels='isFraud', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

In [14]:
lr = LogisticRegression()#class_weight='balanced')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)*100

99.8075

### Now pick a model of your choice and evaluate its accuracy.

In [19]:
# Your code here
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtc.score(X_test, y_test)*100

99.9545

### Which model worked better and how do you know?

In [2]:
# Your response here
'''Decision Tree was better because it got a better score.'''

### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.